### Python Unbox Coding Challenge
#### Author: Subhamoy Bhaduri
#### Date: June 23, 2020

#### Import the libraries.

In [127]:
import random
import numpy as np
from numpy.random import choice
import pandas as pd

#### Set the parameters.

In [128]:
init_population = 2000
crossover_frac = 0.5
mutation_probability = 0.01
target_length = 8 #since our desired output is 8 byte long
cross_over_point = int(crossover_frac*target_length)
generation_count = 5000

population_data = []
fitness_data = []
secure_random = random.SystemRandom()

#### Set additional parameters.

In [129]:
gen_list = np.arange(0, 8) #take all the numbers from the number system between 0 and 7
num_list = [str(i) for i in gen_list] #treat the numbers as Strings to be formatted later
target_list = [str(i) for i in np.arange(0, 8)] #any number between 0 and 7 is part of output sequence
print("List of Numbers in Number System:", num_list)

List of Numbers in Number System: ['0', '1', '2', '3', '4', '5', '6', '7']


#### Create an initial population of different combinations of numbers from Number System.

In [130]:
def Create_Init_Population():  
    population_data = []
    fitness_data = []
    secure_random = random.SystemRandom()
    for outloop in range(init_population):
        random_data = []
        fitness_score = 0
        for inloop in range(target_length):
            selected_data = secure_random.choice(num_list) #pick any number from Number System
            if (selected_data in target_list): #if the number lies between 0 and 7 (our desired pool of numbers) increase the fitness by 1
                fitness_score = fitness_score + 1
            random_data.append(selected_data)
        population_data.append(random_data)
        fitness_data.append(fitness_score)
    probability_dist = []
    for outloop in range(init_population):
        probability_dist.append(fitness_data[outloop]/target_length)
    prob_df = pd.DataFrame({'String':population_data,'Fitness_Score':fitness_data,'Probability':probability_dist})
    prob_df = prob_df.sort_values(['Probability'],ascending=False)
    prob_df = prob_df.reset_index(drop=True)
    return prob_df, population_data

#### Create a function to compute fitness score.

In [131]:
def Compute_Fitness_Score(data):
    data = ''.join([elem for elem in data])
    fitness_score = 0
    board = np.zeros((target_length, target_length))
    for inloop in range(target_length):
#        if (data[inloop] in target_list): #if the number lies between 0 and 7 (our desired pool of numbers)
        if data.count(data[inloop]) == 1: #if there is exact 1 occurance of each number
            #check for diagonality
            row = int(data[inloop])
            col = int(inloop)
            if board[row, col] != 1:
                fitness_score = fitness_score + 1 #increase the fitness by 1
            board[row, :] = 1
            board[:, col] = 1
            #for top right
            for i, j in zip(range(row-1, -1, -1), range(col+1, target_length, 1)):
                board[i, j] = 1
            #for bottom right
            for i, j in zip(range(row+1, target_length, 1), range(col+1, target_length, 1)):
                board[i, j] = 1
    return fitness_score

#### Create a function for formatting the output.

In [132]:
def View_Element(data):
    data = ''.join([elem for elem in data])
    return data

#### Crossover and Mutation.

In [133]:
def Crossover_Mutation(prob_df, population_data):
    for loop in range(generation_count):
        draw=[]
        draw.append(prob_df[0:1]["String"].values[0])
        draw.append(prob_df[1:2]["String"].values[0])
        if (Compute_Fitness_Score(draw[0]) == target_length | Compute_Fitness_Score(draw[1]) == target_length):
            print(View_Element(draw[0]),' ',View_Element(draw[1])) #when desired output is found out
            res = str(View_Element(draw[0]))
            res_fmt = str(' '.join(res))
            return res_fmt
            break
        child1 = draw[0][0:cross_over_point]+draw[1][cross_over_point:] #crossover
        child2 = draw[1][0:cross_over_point]+draw[0][cross_over_point:]
        
        child1_1st = child1[0:cross_over_point]
        child2_1st = child2[0:cross_over_point]
        mutation_list1 = num_list
        mutation_list2 = num_list
        mutation_list1 = [elem for elem in mutation_list1 if elem not in child1_1st]
        mutation_list2 = [elem for elem in mutation_list2 if elem not in child2_1st]
        child1[random.randint(0,target_length-1)] = secure_random.choice(mutation_list1) #mutation
        child2[random.randint(0,target_length-1)] = secure_random.choice(mutation_list2)
        
        population_data.append(child1) #add the newly generated strings back to total population
        population_data.append(child2) #add the newly generated strings back to total population

        fitness_data = []
        total_population = len(population_data)
        for outloop in range(total_population):
            fitness_score = Compute_Fitness_Score(population_data[outloop])
            fitness_data.append(fitness_score)
        probability_dist = []
        for outloop in range(total_population):
            probability_dist.append(fitness_data[outloop]/sum(fitness_data))
        prob_df = pd.DataFrame({'String':population_data,'Fitness_Score':fitness_data,'Probability':probability_dist})
        prob_df = prob_df.sort_values(['Probability'],ascending=False)
        prob_df = prob_df.reset_index(drop=True)
        print('Generation ',loop,' ',' Average Fitness Score ',prob_df["Fitness_Score"].mean(),' ', ''.join(elem for elem in child1),' ',Compute_Fitness_Score(child1),''.join(elem for elem in child2),Compute_Fitness_Score(child2))

#### Define main function.

In [134]:
def main(): 
    probability_df, pop_data = Create_Init_Population() #initial population
    qconfig = Crossover_Mutation(probability_df, pop_data) #calculate qconfig
    print("qconfig: ", qconfig)
      
if __name__=="__main__": 
    main() 

Generation  0    Average Fitness Score  2.4725274725274726   13025130   2 24642255 1
Generation  1    Average Fitness Score  2.473552894211577   60471427   4 53601024 3
Generation  2    Average Fitness Score  2.47357926221336   20766031   3 72554347 2
Generation  3    Average Fitness Score  2.4741035856573705   72551627   2 53206031 4
Generation  4    Average Fitness Score  2.4746268656716417   72052506   2 23746001 4
Generation  5    Average Fitness Score  2.474652087475149   72651427   3 53606131 2
Generation  6    Average Fitness Score  2.477159880834161   30472506   4 23741526 6
Generation  7    Average Fitness Score  2.4801587301587302   23742506   6 23740526 5
Generation  8    Average Fitness Score  2.4801783944499505   23741627   3 23602506 2
Generation  9    Average Fitness Score  2.480693069306931   20361521   4 23744107 2
Generation  10    Average Fitness Score  2.4821958456973294   73552506   4 23746001 4
Generation  11    Average Fitness Score  2.4851778656126484   23741516